Use this for playcounts: https://github.com/entriphy/sp-playcount-librespot

In [ ]:
from datetime import datetime, timedelta
import json
import requests
import random
import pandas as pd
import time

In [ ]:
track_ids = [l.strip() for l in open('./track_ids.txt', 'r').readlines()]
df = pd.DataFrame({'track_id': track_ids})

In [ ]:
spotify_client_id = ''
spotify_client_secret = ''

token = requests.post(
    'https://accounts.spotify.com/api/token',
    data={'grant_type': 'client_credentials'},
    auth=requests.auth.HTTPBasicAuth(
        spotify_client_id,
        spotify_client_secret
    )
).json()

def get_track_info(track_id):
    track_resp = requests.get(
        f'https://api.spotify.com/v1/tracks/{track_id}',
        headers={'Authorization': f'Bearer {token["access_token"]}'}
    )
    time.sleep(.5)
    playcount_resp = requests.get(
        f'https://api.t4ils.dev/albumPlayCount',
        params={'albumid': track_resp.json()["album"]["id"]}
    )
    for album_track in sum((d['tracks'] for d in playcount_resp.json()['data']['discs']), start=[]):
        if track_id in album_track['uri']:
            return album_track

In [ ]:
df['track_info'] = df['track_id'].apply(get_track_info)

In [ ]:
df['name'] = df['track_info'].apply(lambda x: x['name'])
df['artists'] = df['track_info'].apply(lambda x: [a['name'] for a in x['artists']])
df['playcount'] = df['track_info'].apply(lambda x: x['playcount'])
df['explicit'] = df['track_info'].apply(lambda x: x['explicit'])
df['popularity'] = df['track_info'].apply(lambda x: x['popularity'])

In [ ]:
df.to_csv('track_data.csv', index=False)

In [ ]:
sorted_df = df[~df['explicit']].sort_values(['playcount'], ascending=False).reset_index(drop=True)
l1 = list(sorted_df[sorted_df.index % 2 == 0]['track_id'])
l2 = list(sorted_df[sorted_df.index % 2 == 1]['track_id'])
if len(l1) > len(l2):
    del l1[random.randint(0, len(l1)-1)]

matchups = []
while len(l1) > 0:
    i = random.randint(0, min(len(l1) - 1, 9))
    j = random.randint(0, min(len(l2) - 1, 9))
    matchups.append((l1[i], l2[j]) if random.random() > 0.5 else (l2[j], l1[i]))
    del l1[i]
    del l2[j]

matchups.sort(key=lambda x: random.random())

In [ ]:
d = datetime.date(datetime.strptime('2023-12-01', r'%Y-%m-%d'))

schedule = []
for t1id, t2id in matchups:
    schedule.append({
        'date': str(d),
        'track1Id': t1id,
        'track2Id': t2id,
    })
    d += timedelta(days=1)

with open('schedule.json', 'w') as f:
    json.dump(schedule, f)